In [2]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

import numpy as np
from sklearn import *
from lstm.imdb_lstm import *

import matplotlib.pyplot as plt
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Using TensorFlow backend.


# Load Dataset
We reload the dataset with the plain text plots and the labels that reef generated

In [8]:
dataset='imdb'

from data.imdb_loader import DataLoader
# from data.youtube_loader import DataLoader
# from data.trec_loader import DataLoader
# from data.sms_loader import DataLoader

dl = DataLoader()
train_primitive_matrix, val_primitive_matrix, test_primitive_matrix, train_ground, val_ground, \
test_ground, _, _, _, train_text, val_text, test_text = dl.load_data(dataset=dataset, split_val = 0.1)
# _, _, _, train_ground, val_ground, test_ground, train_text, val_text, test_text = dl.load_data(dataset=dataset)
# _, _, _, train_ground, val_ground, test_ground, _,_,_,train_text, val_text, test_text= dl.load_data(dataset=dataset)

len(labels 1920


In [4]:
fname = 'LFs/'+dataset+'/val_5_dict_dt1/normal_reef.npy'
train_reef = np.load(fname)

In [5]:
train_ground.shape, test_ground.shape, val_ground.shape

((1278,), (500,), (142,))

# Train an LSTM Model
We now train a simple LSTM model with the labels generated by Reef. The following hyperparameter search is simplistic, and a more fine-tuned search and a more complex model can improve performance!

__Note that this takes ~1 hour to run on CPU__

In [6]:
train_reef.shape

(1278,)

In [12]:
from get_embeddings import sentences_to_elmo_sentence_embs
test_emb = sentences_to_elmo_sentence_embs(test_text)

train_emb = sentences_to_elmo_sentence_embs(train_text)
val_emb = sentences_to_elmo_sentence_embs(val_text)

W0508 22:21:32.023449 140449556232000 deprecation_wrapper.py:119] From /home/ayusham/auto_lfs/reef/get_embeddings.py:15: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0508 22:21:32.024494 140449556232000 deprecation_wrapper.py:119] From /home/ayusham/auto_lfs/reef/get_embeddings.py:15: The name tf.logging.ERROR is deprecated. Please use tf.compat.v1.logging.ERROR instead.

100%|██████████| 3/3 [00:15<00:00,  5.05s/it]


In [13]:
test_emb[0]

array([ 0.2047744 , -0.21600091, -0.11366825, ..., -0.20995614,
        0.41402677,  0.11663462], dtype=float32)

In [14]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader


In [12]:
# #Label Processing
# train_ground[train_ground == -1] = 0
# test_ground[test_ground == -1] = 0
# max_sentence_length=500
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(train_text)
# tokenizer.fit_on_texts(test_text)
# X_train = tokenizer.texts_to_sequences(train_text)
# X_test = tokenizer.texts_to_sequences(test_text)

In [14]:
# X_train = sequence.pad_sequences(X_train, maxlen=max_sentence_length)
# X_test = sequence.pad_sequences(X_test, maxlen=max_sentence_length)

In [94]:
len(X_train[0])

1024

In [42]:
#top_words=5000
# num_epochs = 30
# batch_size = 32
# learning_rate = 0.001
embedding_vector_length = 1024
# vocab_size=len(tokenizer.word_index) + 1
#LR Model Architecture

num_classes = len(np.unique(test_ground))
print('num classes are ', num_classes)

train_reef = torch.Tensor(train_reef)
test_ground = torch.Tensor(test_ground)
X_train = torch.Tensor(train_primitive_matrix)
X_test = torch.Tensor(test_primitive_matrix)
train_dataset = TensorDataset(X_train, train_reef)
test_dataset = TensorDataset(X_test, test_ground)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)



num classes are  2


### Validation Performance

In [48]:
num_epochs = 50
batch_size = 32
learning_rate = 0.01
n_hidden = 512
n_classes = num_classes
n_features = train_primitive_matrix.shape[1]

In [72]:
import torch.nn.functional as F
class DeepNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DeepNet, self).__init__()
        self.linear_1 = nn.Linear(input_size, hidden_size)
        self.linear_2 = nn.Linear(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = F.sigmoid(self.linear_1(x))
        out = F.sigmoid(self.linear_2(out))
        out = self.out(out)
        out = F.softmax(out, dim=1)
        return out

In [71]:
train_ground.shape

(1278,)

In [61]:
test_ground[test_ground == -1] = 0

In [75]:
# embed = nn.Embedding(vocab_size, embedding_vector_length)

model = DeepNet(n_features, n_hidden, n_classes) #nn.Linear(embedding_vector_length, 2)
# softmax = nn.Softmax()
# criterion = nn.BCEWithLogitsLoss()  
criterion = nn.BCELoss()
# criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (txt, labels) in enumerate(train_loader):
        
        # Reshape images to (batch_size, input_size)
#         txt = txt.reshape(-1, embedding_vector_length)
        
        # Forward pass
#         txt = torch.tensor(txt).to(torch.int64)
#         print(txt.shape)
#         txt1 = embed(txt)
        outputs = model(txt)
#         print(outputs.shape)
#         print(outputs)
        outputs,_ = torch.max(outputs, dim=1)
#         print(outputs, labels)
        
#         print(outputs[0].shape)
        labels = torch.Tensor(labels).double()
        outputs = torch.Tensor(outputs).double()
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         if i%50==0:
#     print('loss ',loss.item())
#             y_pred = model.predict_proba(test_embed)
#             y_pred = np.argmax(y_pred, axis=1)
#             predictions = np.round(y_pred)
#             correct += (predicted == test_labels).sum()
#             print('Accuracy: ', (correct/len(test_labels)))
    correct = 0
    total = 0
    #     for txt, labels in test_loader:
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
#     if num_epochs %10 ==0 :
#         print(predicted)
#     predicted[np.where(predicted==-1)] = 0
    total += test_ground.size(0)
#             print(predicted)
    correct += (predicted == test_ground).float().sum()
#             print('Accuracy %.2f'%(100*correct/total).item())
    print ('Test F1 Score: ', metrics.f1_score(test_ground, predicted, average='macro'))
#             print ('Test Precision: ', metrics.precision_score(test_ground, predicted, average='macro'))
#             print ('Test Recall: ', metrics.recall_score(test_ground, predicted, average='macro'))

#     if (i+1) % 100 == 0:
    print ('Epoch [{}/{}], Loss: {:.4f}' 
           .format(epoch+1, num_epochs, loss.item()))


Test F1 Score:  0.34469200524246396
Epoch [1/50], Loss: 0.6959
Test F1 Score:  0.32157394843962006
Epoch [2/50], Loss: 0.6937
Test F1 Score:  0.34469200524246396
Epoch [3/50], Loss: 0.6932
Test F1 Score:  0.34469200524246396
Epoch [4/50], Loss: 0.6931
Test F1 Score:  0.3676268562332366
Epoch [5/50], Loss: 0.6934
Test F1 Score:  0.34469200524246396
Epoch [6/50], Loss: 0.6935
Test F1 Score:  0.32157394843962006
Epoch [7/50], Loss: 0.6936
Test F1 Score:  0.3937721586728209
Epoch [8/50], Loss: 0.6932
Test F1 Score:  0.34469200524246396
Epoch [9/50], Loss: 0.6934
Test F1 Score:  0.32157394843962006
Epoch [10/50], Loss: 0.6933
Test F1 Score:  0.32157394843962006
Epoch [11/50], Loss: 0.6932
Test F1 Score:  0.34469200524246396
Epoch [12/50], Loss: 0.6932
Test F1 Score:  0.3438320209973753
Epoch [13/50], Loss: 0.6932
Test F1 Score:  0.32157394843962006
Epoch [14/50], Loss: 0.6933
Test F1 Score:  0.34469200524246396
Epoch [15/50], Loss: 0.6933
Test F1 Score:  0.32157394843962006
Epoch [16/50], L

### Test Performance
We re-train the model with the best validation performance since we don't save weights for the models currently.

In [45]:
# with torch.no_grad():
correct = 0
total = 0
#     for txt, labels in test_loader:
outputs = model(X_test)
_, predicted = torch.max(outputs.data, 1)

predicted[np.where(predicted==0)] = -1
total += test_ground.size(0)
correct += (predicted == test_ground).float().sum()
print('Accuracy %.2f'%(100*correct/total).item())
print ('Test F1 Score: ', metrics.f1_score(test_ground, predicted, average='macro'))
print ('Test Precision: ', metrics.precision_score(test_ground, predicted))
print ('Test Recall: ', metrics.recall_score(test_ground, predicted))

RuntimeError: size mismatch, m1: [500 x 176], m2: [1024 x 512] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:136

In [ ]:
print ('Test F1 Score: ', metrics.f1_score(test_ground, predictions, average='macro'))
print ('Test Precision: ', metrics.precision_score(test_ground, predictions))
print ('Test Recall: ', metrics.recall_score(test_ground, predictions))

## [Optional] Ground Truth Performance
We can also train the same model with ground truth labels for the train set to see how far Reef labels are from the best possible performance.

In [ ]:
y_pred = lstm_simple(train_text, train_ground, test_text, test_ground, bs=128, n=10)
predictions = np.round(y_pred)

### Test Performance

In [ ]:
print ('Test F1 Score: ', metrics.f1_score(test_ground, predictions, average='macro'))
print ('Test Precision: ', metrics.precision_score(test_ground, predictions))
print ('Test Recall: ', metrics.recall_score(test_ground, predictions))